## __target__ __update__ Construct

### Simple __target__ __data__ and __target__ __update__ Constructs

The following example shows how the __target__ __update__ construct updates  variables in a device data environment.

The __target__ __data__ construct maps array sections  _v1[:N]_  and  _v2[:N]_   (arrays  _v1_  and  _v2_  in the Fortran code) into a device data environment.

The task executing on the host device encounters the first __target__ region  and waits for the completion of the region.

After the execution of the first __target__ region, the task executing on  the host device then assigns new values to  _v1[:N]_  and  _v2[:N]_  ( _v1_  and  _v2_  arrays  in Fortran code) in the task's data environment by calling the function __init_again()__.

The __target__ __update__ construct assigns the new values of  _v1_  and   _v2_  from the task's data environment to the corresponding mapped array sections  in the device data environment of the __target__ __data__ construct.

The task executing on the host device then encounters the second __target__  region and waits for the completion of the region.

The second __target__ region uses the updated values of  _v1[:N]_  and  _v2[:N]_ .

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: target_update.1
* type: C
* version: omp_4.0
*/
extern void init(float *, float *, int);
extern void init_again(float *, float *, int);
extern void output(float *, int);
void vec_mult(float *p, float *v1, float *v2, int N)
{
   int i;
   init(v1, v2, N);
   #pragma omp target data map(to: v1[:N], v2[:N]) map(from: p[0:N])
   {
      #pragma omp target
      #pragma omp parallel for
      for (i=0; i<N; i++)
 p[i] = v1[i] * v2[i];
      init_again(v1, v2, N);
      #pragma omp target update to(v1[:N], v2[:N])
      #pragma omp target
      #pragma omp parallel for
      for (i=0; i<N; i++)
 p[i] = p[i] + (v1[i] * v2[i]);
   }
   output(p, N);
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: target_update.1
! type: F-free
! version:     omp_4.0
subroutine vec_mult(p, v1, v2, N)
   real    ::  p(N), v1(N), v2(N)
   integer ::  i
   call init(v1, v2, N)
   !$omp target data map(to: v1, v2) map(from: p)
      !$omp target
      !$omp parallel do
         do i=1,N
            p(i) = v1(i) * v2(i)
         end do
      !$omp end target
      call init_again(v1, v2, N)
      !$omp target update to(v1, v2)
      !$omp target
      !$omp parallel do
         do i=1,N
            p(i) = p(i) + v1(i) * v2(i)
         end do
      !$omp end target
   !$omp end target data
   call output(p, N)
end subroutine

### __target__ __update__ Construct with __if__ Clause

The following example shows how the __target__ __update__ construct updates  variables in a device data environment.

The __target__ __data__ construct maps array sections  _v1[:N]_  and  _v2[:N]_   (arrays  _v1_  and  _v2_  in the Fortran code) into a device data environment. In between  the two __target__ regions, the task executing on the host device conditionally  assigns new values to  _v1_  and  _v2_  in the task's data environment. The function __maybe_init_again()__  returns  _true_  if new data is written.

When the conditional expression (the return value of __maybe_init_again()__) in the  __if__ clause is  _true_ , the __target__ __update__ construct  assigns the new values of  _v1_  and  _v2_  from the task's data environment to the corresponding  mapped array sections in the __target__ __data__ construct's device data  environment.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: target_update.2
* type: C
* version: omp_4.0
*/
extern void init(float *, float *, int);
extern int maybe_init_again(float *, int);
extern void output(float *, int);
void vec_mult(float *p, float *v1, float *v2, int N)
{
   int i;
   init(v1, v2, N);
   #pragma omp target data map(to: v1[:N], v2[:N]) map(from: p[0:N])
   {
      int changed;
      #pragma omp target
      #pragma omp parallel for
      for (i=0; i<N; i++)
        p[i] = v1[i] * v2[i];
      changed = maybe_init_again(v1,  N);
      #pragma omp target update if (changed) to(v1[:N])
      changed = maybe_init_again(v2,  N);
      #pragma omp target update if (changed) to(v2[:N])
      #pragma omp target
      #pragma omp parallel for
      for (i=0; i<N; i++)
        p[i] = p[i] + (v1[i] * v2[i]);
   }
   output(p, N);
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: target_update.2
! type: F-free
! version:     omp_4.0
subroutine vec_mult(p, v1, v2, N)
   interface
      logical function maybe_init_again (v1, N)
      real :: v1(N)
      integer :: N
      end function
   end interface
   real    ::  p(N), v1(N), v2(N)
   integer ::  i
   logical :: changed
   call init(v1, v2, N)
   !$omp target data map(to: v1, v2) map(from: p)
      !$omp target
         !$omp parallel do
         do i=1, N
            p(i) = v1(i) * v2(i)
         end do
      !$omp end target
      changed = maybe_init_again(v1, N)
      !$omp target update if(changed) to(v1(:N))
      changed = maybe_init_again(v2, N)
      !$omp target update if(changed) to(v2(:N))
      !$omp target
         !$omp parallel do
         do i=1, N
            p(i) = p(i) + v1(i) * v2(i)
         end do
      !$omp end target
   !$omp end target data
   call output(p, N)
end subroutine